<a href="https://colab.research.google.com/github/bhowad-akash/Machine-Learning/blob/main/Convolutional_Neural_Network_Architectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Typical CNN Architectures**

Below-mentioned are a few well-known CNN architectures (for image
classification, but can be adapted for other applications)


*   AlexNet
*   GoogLeNet (a.k.a. InceptionNet)
*   VGGNet
*   ResNet
*   Xception

They often involve new ideas for training, and new modules that can
be reused in new architectures.

Training these models can take a large amount of resources.

Keras supports typical models with pre-trained weights on ImageNet
E.g. Xception, VGG16, VGG19, ResNet, ResNetV2, InceptionV3, etc.

Refer to the https://keras.io/applications/ for more details.
These are for classifying images (into 1,000 categories)

In order to classify the other images, these Pre trained CNN models could be used.

## ***Image Classification using Pre-Trained Networks***

In [1]:
#Import necessary libraries
import tensorflow as tf
import keras

In [2]:
#Loading Pre-Trained Resnet50 Model
model = keras.applications.ResNet50(weights='imagenet')

102967424/102967424 [==============================] - 1s 0us/step


In [3]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [4]:
#Importing Keras module that contains utility functions for handling images
import keras.utils as image

#Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
'''The function to load image from the specified path
The image is resized to 224x224 pixels, which is the input size required by the ResNet50 model'''

img = image.load_img("/content/drive/MyDrive/Applied Machine Learning/Week 8/IndianElephant.jpg", target_size=(224, 224))

In [6]:
#Converting image to Numpy array (common format for image data in deep learning)
img = image.img_to_array(img)
img.shape

(224, 224, 3)

In [7]:
#Reshaping data for the model

'''The ResNet50 model, like many other convolutional neural networks (CNNs),
expects the input to be in the form of a 4D tensor with the shape (batch_size, height, width, channels)

After loading and converting the image to a NumPy array,
the shape is (height, width, channels), which in this case would be (224, 224, 3).

The model expects a shape of (batch_size, height, width, channels).
For a single image, the batch size is 1, so the expected shape is (1, 224, 224, 3)'''

img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
img.shape

(1, 224, 224, 3)

In [8]:
#Pre-processing the image for the model

'''In order to match the format expected by the ResNet50 model expects,
we need to pereform the pre processing step.

This step esentially involves scaling pixel values and
performing any other model-specific preprocessing.'''

img = keras.applications.resnet50.preprocess_input(img)

In [9]:
#Performing Predictions

'''Uses the pre-trained model to predict the class probabilities for the input image.
Y_prob is an array of probabilities for each class.'''

Y_prob = model.predict(img)

1/1 [==============================] - 4s 4s/step


In [10]:
#Top K Predictions

'''Converts the model's output probabilities into human-readable class labels
and their associated probabilities.'''

top_k = keras.applications.resnet50.decode_predictions(Y_prob, top=3)

35363/35363 [==============================] - 0s 0us/step


In [11]:
'''Iterates through the top 3 predictions and prints the class ID,
class name, and the probability (converted to a percentage)'''

for class_id, name, y_proba in top_k[0]:
  print("class_id:", class_id, "name:", name, " ", y_proba*100, "%")

class_id: n02504458 name: African_elephant   61.43888235092163 %
class_id: n01871265 name: tusker   27.420103549957275 %
class_id: n02504013 name: Indian_elephant   10.92836931347847 %


## ***Fine-Tuning Pre-Trained Networks***

Although pre-trained models are for a different purpose, it is likely
that the learned weights can be useful for other applications.

**Transfer Learning**

Take a pre-trained model

*   Take a pre-trained model
*   Remove the top layer(s) (as it is related to the specific task)
*   Add new layers based on the need of the application
*   Train the model with existing layer weights frozen
*   Unfreeze all layers and train the whole model

Benefits from existing models which are trained on a large amount of
data for a different purpose (where data is more limited)

In [12]:
#Importing Tensorflow Datasets

'''TensorFlow Datasets provides a collection of ready-to-use datasets
for use with TensorFlow, including images, text, video, and audio'''

import tensorflow_datasets as tfds

In [13]:
'''Loading the TF Flowers dataset, in the supervised learning format (tuple (input, label))
and the dataset information (metadata)'''

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [14]:
#Size of the number of examples in the training split
dataset_size = info.splits['train'].num_examples
print('Size:', dataset_size)

Size: 3670


In [15]:
#Class Names
class_names = info.features['label'].names
print('Classes: ', class_names)

Classes:  ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']


In [16]:
#No. of Classes
n_classes = info.features['label'].num_classes
print('No. of Classes: ', n_classes)

No. of Classes:  5


In [17]:
#Splitting the data into Train, Test and Validation

'''Setting as Supervised to True to include Ground Truth Labels'''

train_set = tfds.load('tf_flowers', split='train[:75%]', as_supervised=True)
test_set = tfds.load('tf_flowers', split='train[75%:90%]', as_supervised=True)
valid_set = tfds.load('tf_flowers', split='train[90%:]', as_supervised=True)

In [18]:
#Data preprocessing
def preprocess(image, label):
  resized_image = tf.image.resize(image, [224, 224])  #Resizing image to 224x224 pixels as. This is a common size used in many pre-trained models (e.g., Xception, ResNet, VGG) because they were trained on images of this size.
  final_image = keras.applications.xception.preprocess_input(resized_image) #Function applying necessary pre processing specific to Xception model, usually involves scaling pixel values to the range expected by the model (e.g., from [0, 255] to [-1, 1]).
  return final_image, label

batch_size = 32
train_set = train_set.shuffle(1000)   #Shuffling the dataset to reduce model overfitting

'''The map function applies the preprocess function to each image and label in the dataset.

The repeat function makes the dataset iterate indefinitely, This is useful during training because
it allows the model to go through multiple epochs without manually re-initializing the dataset.

The batch function groups the dataset into batches of size 32.
Batching helps in efficient training by allowing the model to process multiple samples simultaneously.

Prefetching overlaps the preprocessing and model execution of the next batch,
improving performance by ensuring that data is ready when the model is done processing the current batch.'''

train_set = train_set.map(preprocess).repeat().batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).repeat().batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)

In [19]:
#Loading and fine tuning the Pre-trained Xception Model

'''The model is loaded with weights pre-trained on the ImageNet dataset.
The top layers are excluded because we want to replace them with layers suitable for our specific task'''

base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)

'''Add a global average pooling layer followed by a dense layer with a softmax activation
to adapt the model for our specific classification task'''

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.Model(inputs=base_model.input, outputs=output)
model.summary()

83683744/83683744 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, None, None, 32)       864       ['input_2[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, None, None, 32)       128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  

In [20]:
#Training with existing layers fixed

'''the pre-trained layers retain their learned features while the new layers are specifically trained
for the flower classification task, leading to better generalization and performance.'''

for layer in base_model.layers:
  layer.trainable = False

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.02,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

history = model.fit(train_set, epochs=5, steps_per_epoch=dataset_size*0.75//batch_size, validation_data=valid_set, validation_steps = dataset_size*0.15//batch_size)

Epoch 1/5
86/86 [==============================] - 27s 193ms/step - loss: 0.5468 - accuracy: 0.7911 - val_loss: 0.4107 - val_accuracy: 0.8713
Epoch 2/5
86/86 [==============================] - 13s 153ms/step - loss: 0.2919 - accuracy: 0.8950 - val_loss: 0.3445 - val_accuracy: 0.8640
Epoch 3/5
86/86 [==============================] - 14s 159ms/step - loss: 0.2274 - accuracy: 0.9226 - val_loss: 0.3318 - val_accuracy: 0.8824
Epoch 4/5
86/86 [==============================] - 14s 159ms/step - loss: 0.1844 - accuracy: 0.9415 - val_loss: 0.2939 - val_accuracy: 0.8897
Epoch 5/5
86/86 [==============================] - 13s 154ms/step - loss: 0.1685 - accuracy: 0.9495 - val_loss: 0.3168 - val_accuracy: 0.8805


In [21]:
# The model can be further trained with base layers unfrozen

'''Fine-tuning All Layers: Initially, only the new classification layers were trained,
while the pre-trained layers were frozen. This approach is often taken to avoid overfitting
and to quickly adapt the model to a new task.

Once the new layers have been trained, unfreezing all layers allows the model to fine-tune the entire network,
which can lead to better performance since the model can adjust the pre-trained features more precisely to the new task.'''

for layer in base_model.layers:
  layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=lr_schedule, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(train_set, epochs=5, steps_per_epoch=dataset_size*0.75//batch_size, validation_data=valid_set, validation_steps = dataset_size*0.15//batch_size)

Epoch 1/5
86/86 [==============================] - 63s 508ms/step - loss: 0.3262 - accuracy: 0.8866 - val_loss: 0.4743 - val_accuracy: 0.8695
Epoch 2/5
86/86 [==============================] - 42s 491ms/step - loss: 0.0953 - accuracy: 0.9695 - val_loss: 0.5873 - val_accuracy: 0.8897
Epoch 3/5
86/86 [==============================] - 42s 492ms/step - loss: 0.0453 - accuracy: 0.9862 - val_loss: 0.1863 - val_accuracy: 0.9338
Epoch 4/5
86/86 [==============================] - 43s 495ms/step - loss: 0.0167 - accuracy: 0.9945 - val_loss: 0.1331 - val_accuracy: 0.9485
Epoch 5/5
86/86 [==============================] - 42s 488ms/step - loss: 0.0112 - accuracy: 0.9960 - val_loss: 0.1578 - val_accuracy: 0.9522


In [22]:
#Evaluating the Model
test_loss, test_accuracy = model.evaluate(test_set, steps=dataset_size*0.10//batch_size)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

11/11 [==============================] - 2s 123ms/step - loss: 0.2414 - accuracy: 0.9347
Test Loss: 0.24136807024478912
Test Accuracy: 0.9346590638160706
